In [1]:
!pip install transformers datasets torch
!pip install accelerate -U
!pip install transformers[torch] --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (12

In [21]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import pandas as pd
import accelerate
import transformers
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
!ls "/content/drive/MyDrive/gpt-finetuning/data/quotes"


quotes.csv  test.txt  train.txt  valid.txt


In [5]:
# Load data
df = pd.read_csv("/content/drive/MyDrive/gpt-finetuning/data/quotes/quotes.csv")
df.head()

,quote,author,category
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."
3,A friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love"
4,Darkness cannot drive out darkness: only light...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh..."


In [13]:
df_tiny = df.sample(frac=0.02)
df_tiny.shape

(9994, 3)

In [14]:
# Create a text field combining category tags and quotes
df_tiny['text'] = df_tiny['category'] + ": " + df_tiny['quote']

# Convert to Dataset
dataset = Dataset.from_pandas(df_tiny)

# Split the dataset into train and test
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [24]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# Load GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

In [16]:
def fill_none_values(examples):
    examples['text'] = ["<empty>" if text is None else text for text in examples['text']]
    return examples

In [25]:
# Tokenization
def tokenize_function(examples):
  examples = fill_none_values(examples)
  tokenized_output = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128, return_tensors='pt')
  tokenized_output['labels'] = tokenized_output['input_ids'].clone()
  return tokenized_output

tokenizer.pad_token = tokenizer.eos_token
# Apply the tokenization function to the datasets using map
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/8994 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,1.419000,1.401733


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=4497, training_loss=1.4734364467804926, metrics={'train_runtime': 608.7964, 'train_samples_per_second': 14.773, 'train_steps_per_second': 7.387, 'total_flos': 587515133952000.0, 'train_loss': 1.4734364467804926, 'epoch': 1.0})

In [ ]:
# Save the model
trainer.save_model("finetuned_gpt2_model")

# Experiment 1: Temperature
---

In [46]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=1.5, top_k=50, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,   632,   561,   407,
           772,   307,  3148,   284,   307,   616,  1545,   611,   345,  6304,
           470,  1541,  1972,   340,   572,   616, 23179, 50256]])
motivation, friend, success: It would not even be fair to be my friend if you weren't already getting it off my sleeve


In [47]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=1.0, top_k=50, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,  1002,   345,   821,
          2045,   284,   307,   257,   922,  1545,    11,   345,   760,   345,
           761,   284,   670,  1327,   523,   345,   460,   787,   534,  1545,
          1254,  6792,    13,   632,   338,   257,  7947,   345,   460,  2107,
            13, 50256]])
motivation, friend, success: If you're looking to be a good friend, you know you need to work hard so you can make your friend feel comfortable. It's a habit you can live.


In [49]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=1.2, top_k=50, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,   314,   423,  2147,
          1231,  1854,    13, 50256]])
motivation, friend, success: I have nothing without others.


In [50]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.7, top_k=50, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,   314,   892,   326,
           611,   345,   765,   284,  6758,    11,   345,   761,   284,  1949,
           284,   307,   257,  1545,   286,   883,   508,   423,   587,   612,
           878,    13, 50256]])
motivation, friend, success: I think that if you want to succeed, you need to try to be a friend of those who have been there before.


In [51]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.9, top_k=50, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,  8975,   340,   691,
         20406,   611,   534,  1545,  5300,  5938,    13, 50256]])
motivation, friend, success: Sometimes it only hurts if your friend feels hurt.


# Experiment 2: top_k

In [52]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.9, top_k=25, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,  1002,   345,   765,
           284,   307,  4388,   475,   423,   645,  2460,    11,   345,   423,
           284,   466,  1243,   326,   787,   345,   466,   340,   329,   262,
           749,   636,    13,   921,   423,   284,   466,  1243,   326,   345,
           836,   470,   765,   284,   466,    11,   475,   635,   466,   517,
           780,   345,   765,   284,   307,  4388,    13, 50256]])
motivation, friend, success: If you want to be successful but have no friends, you have to do things that make you do it for the most part. You have to do things that you don't want to do, but also do more because you want to be successful.


In [54]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.9, top_k=50, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,   383,  1266,  1048,
           314,   760,   857,   655,   326,    13,   775,   477,   389,  3748,
           287,   674,   898,  2842,   290,   477,   286,   606,   460,  1487,
           674,  3160,    13, 50256]])
motivation, friend, success: The best person I know does just that. We all are unique in our own ways and all of them can change our lives.


In [55]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.9, top_k=75, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,   921,   821,   281,
           719,   286, 14738,    11,   290,   523,   318,   534,  1545,    13,
           887, 14738,   318,   546,  4917,   262,  1048,  3025,   922, 14953,
           389,   922,   290,   345,   815,   307,  1498,   284,  1210,   606,
           287,    13, 50256]])
motivation, friend, success: You're an act of friendship, and so is your friend. But friendship is about finding the person whose good intentions are good and you should be able to turn them in.


In [56]:
# Load the model for inference
model = GPT2LMHeadModel.from_pretrained("finetuned_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Inference function
def generate_quote(theme, max_length=50):
    input_text = theme + ":"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    print(inputs)
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, temperature=0.9, top_k=100, do_sample=True)
    # num_beams=5,
    print(outputs)
    generated_quote = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_quote)
    return generated_quote

# Example usage
theme = "motivation, friend, success"
generated_quote = generate_quote(theme)

tensor([[27926, 26939,    11,  1545,    11,  1943,    25]])
tensor([[27926, 26939,    11,  1545,    11,  1943,    25,  1002,   345,   389,
          4609,   287,   534,   898,  2614,  1943,    11,   788,   345,   389,
          1365,   572, 10342,   329,  2130,   284,   307,   534,  1545,    13,
         50256]])
motivation, friend, success: If you are interested in your own personal success, then you are better off searching for someone to be your friend.


In [57]:
import os

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if not os.path.islink(file_path):
                total_size += os.path.getsize(file_path)
    return total_size

folder_path = './finetuned_gpt2_model'
folder_size = get_folder_size(folder_path)
print(f"Folder size: {folder_size / (1024 * 1024):.2f} MB")

Folder size: 474.72 MB
